Keras model
=====

here we try to implement the keras model

In [1]:
import pandas as pd
import numpy as np
%cd ../data
data_original = pd.read_csv('data_cleaned.csv',  delimiter=',', header=0)

/home/raul/uni/data-science/ids2019-m16-traffic-violations/data


let's add the features and convert them to categorical

let's 

In [2]:
data_original.head()


data_x = data_original[['vehicle_brand', 'vehicle_year', 'vehicle_type', 'age_group', 'gender']]
data_x['vehicle_brand'] = pd.Categorical(data_x['vehicle_brand']).codes
data_x['vehicle_year'] = pd.to_numeric(data_x['vehicle_year'])
data_x['vehicle_type'] = pd.Categorical(data_x['vehicle_type']).codes
data_x['age_group'] = pd.Categorical(data_x['age_group']).codes
data_x['gender'] = pd.Categorical(data_x['gender']).codes




data_y = pd.Categorical(data_original['infraction']).codes

infraction_map = pd.DataFrame(data_original['infraction'])
infraction_map['infraction_code'] = pd.Categorical(infraction_map.infraction).codes
infraction_map = infraction_map.drop_duplicates().sort_values('infraction_code').reset_index(drop=True)




data_x.age_group.unique()

/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

array([ 3,  8,  7,  4,  6,  5,  2, -1,  1,  0])

x to categorical

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=0)

X_train.shape

(286719, 5)

model time...

In [13]:
import xgboost as xgb
dtrain=xgb.DMatrix(data=X_train, label=y_train)
dtest=xgb.DMatrix(X_test)
params = {
    'max_depth': 3,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 10
}
bst = xgb.train(params, dtrain)
y = bst.predict(dtest)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras import optimizers

model = Sequential([
    Dense(5, input_shape=(X_train.shape[1],)),
    Dense(200,),
    Dense(200,),
    Dense(200),
    Dense(500),

    Dense(1),
    Activation('softmax')
])

sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='huber_loss', metrics=['accuracy'])

#.shape[1]
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
277600/286719 [============================>.] - ETA: 0s - loss: 5.2322 - accuracy: 9.5101e-04

In [10]:
result = pd.DataFrame(model.predict(X_test))

In [14]:
print(classification_report(y_test, y))

              precision    recall  f1-score   support

           0       0.30      0.02      0.03      8431
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00      4879
           3       0.53      0.06      0.10      7073
           4       0.00      0.00      0.00       980
           5       0.50      0.47      0.48      3955
           6       0.00      0.00      0.00       366
           7       0.00      0.00      0.00      6343
           8       0.50      0.98      0.66     45325
           9       0.21      0.02      0.03     18128

    accuracy                           0.49     95574
   macro avg       0.20      0.15      0.13     95574
weighted avg       0.36      0.49      0.35     95574

